# Processo seletivo 2RPNet - Desafio

Queries

Para trabalhar com os dados, devido ao grande volume e falta de um RDBMS, limitei à um numero bem menor de linhas comparado ao todo. Lí poucas linhas com um dataframe Pandas, transformei esse dataframe em um dataframe Spark e rodei rotinas SQL usando Spark.sql.

In [1]:
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

23/05/22 11:29:12 WARN Utils: Your hostname, guilherme-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/22 11:29:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/22 11:29:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/22 11:29:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = pd.read_csv("EPD_202302.ZIP", header=0, nrows=100000)
##ler as 100000 primeiras linhas e armazenar num df Pandas

In [3]:
sparkDF = spark.createDataFrame(df)
##transforma o df pandas em um df spark,
## para ser possível rodar rotinas SQL

In [4]:
sparkDF.printSchema()

root
 |-- YEAR_MONTH: long (nullable = true)
 |-- REGIONAL_OFFICE_NAME: string (nullable = true)
 |-- REGIONAL_OFFICE_CODE: string (nullable = true)
 |-- ICB_NAME: string (nullable = true)
 |-- ICB_CODE: string (nullable = true)
 |-- PCO_NAME: string (nullable = true)
 |-- PCO_CODE: string (nullable = true)
 |-- PRACTICE_NAME: string (nullable = true)
 |-- PRACTICE_CODE: string (nullable = true)
 |-- ADDRESS_1: string (nullable = true)
 |-- ADDRESS_2: string (nullable = true)
 |-- ADDRESS_3: string (nullable = true)
 |-- ADDRESS_4: string (nullable = true)
 |-- POSTCODE: string (nullable = true)
 |-- BNF_CHEMICAL_SUBSTANCE: string (nullable = true)
 |-- CHEMICAL_SUBSTANCE_BNF_DESCR: string (nullable = true)
 |-- BNF_CODE: string (nullable = true)
 |-- BNF_DESCRIPTION: string (nullable = true)
 |-- BNF_CHAPTER_PLUS_CODE: string (nullable = true)
 |-- QUANTITY: double (nullable = true)
 |-- ITEMS: long (nullable = true)
 |-- TOTAL_QUANTITY: double (nullable = true)
 |-- ADQUSAGE: double 

In [4]:
sparkDF.createOrReplaceTempView("dfView")
spark.sql("""SELECT *
FROM dfview;
""").show(5)

23/05/22 11:31:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/05/22 11:31:08 WARN TaskSetManager: Stage 0 contains a task of very large size (15579 KiB). The maximum recommended task size is 1000 KiB.
23/05/22 11:31:15 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


+----------+--------------------+--------------------+--------------------+--------+--------------------+--------+--------------+-------------+--------------------+---------+---------+---------+--------+----------------------+----------------------------+---------------+--------------------+---------------------+--------+-----+--------------+--------+------+-----------+------------+
|YEAR_MONTH|REGIONAL_OFFICE_NAME|REGIONAL_OFFICE_CODE|            ICB_NAME|ICB_CODE|            PCO_NAME|PCO_CODE| PRACTICE_NAME|PRACTICE_CODE|           ADDRESS_1|ADDRESS_2|ADDRESS_3|ADDRESS_4|POSTCODE|BNF_CHEMICAL_SUBSTANCE|CHEMICAL_SUBSTANCE_BNF_DESCR|       BNF_CODE|     BNF_DESCRIPTION|BNF_CHAPTER_PLUS_CODE|QUANTITY|ITEMS|TOTAL_QUANTITY|ADQUSAGE|   NIC|ACTUAL_COST|UNIDENTIFIED|
+----------+--------------------+--------------------+--------------------+--------+--------------------+--------+--------------+-------------+--------------------+---------+---------+---------+--------+----------------------+--

### a. Crie um dataframe contendo os 10 principais produtos químicos prescritos por região.

In [21]:
spark.sql("""
SELECT
    REGIONAL_OFFICE_NAME as regiao,
    CHEMICAL_SUBSTANCE_BNF_DESCR as prod_quimico
FROM (
    SELECT
        REGIONAL_OFFICE_NAME,
        CHEMICAL_SUBSTANCE_BNF_DESCR,
        COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR) as N_PRESCRIPTIONS,
        RANK() OVER (
            PARTITION BY REGIONAL_OFFICE_NAME ORDER BY COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR) DESC
        ) as RANK
    FROM dfview
    GROUP BY REGIONAL_OFFICE_NAME, CHEMICAL_SUBSTANCE_BNF_DESCR
) as rank_f WHERE RANK <= 10;
""").show(20)

23/05/21 22:02:47 WARN TaskSetManager: Stage 28 contains a task of very large size (15579 KiB). The maximum recommended task size is 1000 KiB.


+---------------+--------------------+
|         regiao|        prod_quimico|
+---------------+--------------------+
|EAST OF ENGLAND|      Colecalciferol|
|EAST OF ENGLAND|    Other Appliances|
|EAST OF ENGLAND|           Estradiol|
|EAST OF ENGLAND|        Eye Products|
|EAST OF ENGLAND|Metformin hydroch...|
|EAST OF ENGLAND|Levothyroxine sodium|
|EAST OF ENGLAND|Glucose blood tes...|
|EAST OF ENGLAND|          Gabapentin|
|EAST OF ENGLAND|Co-codamol (Codei...|
|EAST OF ENGLAND|       Macrogol 3350|
|         LONDON|      Colecalciferol|
|         LONDON|Metformin hydroch...|
|         LONDON|Levothyroxine sodium|
|         LONDON|    Other Appliances|
|         LONDON|           Estradiol|
|         LONDON|        Eye Products|
|         LONDON|          Emollients|
|         LONDON|        Prednisolone|
|         LONDON|Glucose blood tes...|
|         LONDON|          Pregabalin|
+---------------+--------------------+
only showing top 20 rows



### b. Quais produtos químicos prescritos tiveram a maior somatória de custos por mês? Liste os 10 primeiros

In [23]:
spark.sql("""
    SELECT
    CHEMICAL_SUBSTANCE_BNF_DESCR as prod_quimico,
    YEAR_MONTH as ano_mes,
    MAX_COST as custo_max
FROM 
(
    SELECT
        CHEMICAL_SUBSTANCE_BNF_DESCR,
        ROUND(MAX(ACTUAL_COST),2) as MAX_COST,
        YEAR_MONTH
    FROM dfview
    GROUP BY CHEMICAL_SUBSTANCE_BNF_DESCR, YEAR_MONTH
)
LIMIT 10;
""").show(10)
## Apesar de estar trabalhando com dados de apenas
## 1 mês, a query acima funcionaria para um dataset
## com mais meses

23/05/21 22:04:16 WARN TaskSetManager: Stage 37 contains a task of very large size (15579 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-------+---------+
|        prod_quimico|ano_mes|custo_max|
+--------------------+-------+---------+
|Other eye tear/lu...| 202302|    14.14|
|           Sterculia| 202302|    12.53|
|          Permethrin| 202302|   111.78|
|Mometasone/glycop...| 202302|   208.65|
|           Mupirocin| 202302|     31.9|
|          Eprosartan| 202302|    67.14|
|        Prucalopride| 202302|   288.01|
|Specialist food r...| 202302|    50.07|
|RtS 2.4 kcal/ml d...| 202302|   129.17|
|             Estriol| 202302|   469.22|
+--------------------+-------+---------+



### c. Quais são as prescrições mais comuns?

In [32]:
spark.sql("""
SELECT BNF_DESCRIPTION as prescricao, COUNT(BNF_DESCRIPTION) as vezes_prescrito
FROM dfview
GROUP BY prescricao
ORDER BY vezes_prescrito DESC
LIMIT 10;
""").show(10)

23/05/21 14:27:47 WARN TaskSetManager: Stage 53 contains a task of very large size (15579 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------------+
|          prescricao|vezes_prescrito|
+--------------------+---------------+
|Paracetamol 500mg...|            479|
|Gabapentin 300mg ...|            466|
|Prednisolone 5mg ...|            443|
|Tramadol 50mg cap...|            391|
|Metformin 500mg t...|            366|
|Fluoxetine 20mg c...|            334|
|Amitriptyline 10m...|            322|
|Codeine 30mg tablets|            319|
|Folic acid 5mg ta...|            317|
|Levothyroxine sod...|            310|
+--------------------+---------------+



### d. Qual produto químico é mais prescrito por cada prescriber?

In [47]:
from pyspark.sql.functions import *
spark.sql("""
select CHEMICAL_SUBSTANCE_BNF_DESCR, PCO_NAME, COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR, PCO_NAME) as vezes
from dfView 
group by CHEMICAL_SUBSTANCE_BNF_DESCR, PCO_NAME
""").\
groupBy("PCO_NAME").agg(first("CHEMICAL_SUBSTANCE_BNF_DESCR"),max("vezes")).show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'dfview'.(line 4, pos 5)

== SQL ==

SELECT CHEMICAL_SUBSTANCE_BNF_DESCR, PCO_NAME, MAX(vezes)
FROM (SELECT CHEMICAL_SUBSTANCE_BNF_DESCR, PCO_NAME, COUNT(CHEMICAL_SUBSTANCE_BNF_DESCR, PCO_NAME) as vezes)
FROM dfview 
-----^^^
GROUP BY CHEMICAL_SUBSTANCE_BNF_DESCR, PCO_NAME
ORDER BY MAX(vezes) DESC


### e. Quantos prescribers foram adicionados no último mês?

In [29]:
spark.sql("""
SELECT DISTINCT PCO_NAME as prescriber
FROM dfview
WHERE YEAR_MONTH LIKE "%%%%02" 
AND NOT EXISTS(
    SELECT PCO_NAME as prescriber
    FROM dfview
    WHERE YEAR_MONTH NOT LIKE "%%%%02"
);
""").show(10)
## Apesar de estar trabalhando com dados de apenas
## 1 mês, a query acima funcionaria para um dataset
## com mais meses

23/05/21 14:24:32 WARN TaskSetManager: Stage 43 contains a task of very large size (15579 KiB). The maximum recommended task size is 1000 KiB.
23/05/21 14:24:35 WARN TaskSetManager: Stage 44 contains a task of very large size (15579 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+
|          prescriber|
+--------------------+
|NHS NOTTINGHAM AN...|
|NHS GREATER MANCH...|
|NHS WEST YORKSHIR...|
|NHS NORTHAMPTONSH...|
|NHS CAMBRIDGESHIR...|
|NHS NORTH EAST AN...|
|NHS BLACK COUNTRY...|
|NHS KENT AND MEDW...|
|NHS CORNWALL AND ...|
|NHS SOUTH WEST LO...|
+--------------------+
only showing top 10 rows



### f. Quais prescribers atuam em mais de uma região? Ordene por quantidade de regiões atendidas.

In [69]:
spark.sql("""
    SELECT PCO_NAME as prescriber, COUNT(DISTINCT PCO_NAME, REGIONAL_OFFICE_NAME) as nro_regioes_atendidas
    FROM dfview
    GROUP BY prescriber
    HAVING nro_regioes_atendidas > 1
    ORDER BY nro_regioes_atendidas DESC
""").show(20)

23/05/21 23:21:40 WARN TaskSetManager: Stage 161 contains a task of very large size (15579 KiB). The maximum recommended task size is 1000 KiB.


+----------+---------------------+
|prescriber|nro_regioes_atendidas|
+----------+---------------------+
+----------+---------------------+



### g. Qual o preço médio dos químicos prescritos no último mês coletado?

In [30]:
spark.sql("""
SELECT CHEMICAL_SUBSTANCE_BNF_DESCR as prod_quimico, ROUND(AVG(ACTUAL_COST), 2) as preco_medio
FROM dfview
WHERE YEAR_MONTH LIKE "%%%%02"
GROUP BY prod_quimico
ORDER BY preco_medio DESC;
""").show(10)
## Apesar de estar trabalhando com dados de apenas
## 1 mês, a query acima funcionaria para um dataset
## com mais meses

23/05/21 14:26:35 WARN TaskSetManager: Stage 47 contains a task of very large size (15579 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----------+
|        prod_quimico|preco_medio|
+--------------------+-----------+
|Other food for sp...|    4016.65|
|        Voriconazole|    3618.99|
|           Diazoxide|    3299.87|
|Powder 20g GMP pr...|    2357.64|
|Apomorphine hydro...|     1631.2|
|Powder 15g GMP pr...|    1574.49|
|             Danazol|    1468.05|
|Apomorphine hydro...|    1323.68|
|          Lanreotide|    1317.62|
|    Calcium folinate|     1063.7|
+--------------------+-----------+
only showing top 10 rows



### h. Gere uma tabela que contenha apenas a prescrição de maior valor de cada usuário.

In [ ]:
## Não entendi o que a solicitação 
## quis dizer com usuário